In [22]:
import os
import sys

# 设置CUDA_HOME环境变量
conda_prefix = os.environ.get('CONDA_PREFIX')
if conda_prefix:
    os.environ['CUDA_HOME'] = conda_prefix
    os.environ['LD_LIBRARY_PATH'] = f"{conda_prefix}/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"
    
    print(f"已设置环境变量:")
    print(f"  CUDA_HOME: {os.environ['CUDA_HOME']}")
    print(f"  CONDA_PREFIX: {conda_prefix}")
    
    # 验证libcuda.so是否存在
    libcuda_path = f"{conda_prefix}/lib/libcuda.so"
    if os.path.exists(libcuda_path):
        print(f"  ✓ libcuda.so: 找到 ({libcuda_path})")
    else:
        print(f"  ✗ libcuda.so: 在conda环境中未找到")
        
        # 创建符号链接到系统libcuda
        system_libcuda = "/usr/lib/x86_64-linux-gnu/libcuda.so"
        if os.path.exists(system_libcuda):
            os.system(f"ln -sf {system_libcuda} {conda_prefix}/lib/libcuda.so")
            print(f"  ✓ 已创建符号链接: {conda_prefix}/lib/libcuda.so -> {system_libcuda}")
        else:
            print(f"  ⚠ 系统libcuda.so也未找到")
else:
    print("CONDA_PREFIX未设置，请先激活conda环境")

已设置环境变量:
  CUDA_HOME: /home/y/anaconda3/envs/mindspore
  CONDA_PREFIX: /home/y/anaconda3/envs/mindspore
  ✓ libcuda.so: 找到 (/home/y/anaconda3/envs/mindspore/lib/libcuda.so)


In [23]:
import os
import sys
import numpy as np
import matplotlib
matplotlib.use('Agg')  # 使用非交互式后端，避免GUI问题
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# ============================================================
# 先导入MindSpore相关模块
# ============================================================
try:
    import mindspore as ms
    from mindspore import context, Tensor, Model, load_checkpoint, save_checkpoint, Parameter
    import mindspore.nn as nn
    import mindspore.ops as ops
    from mindspore.train.callback import LossMonitor, TimeMonitor, ModelCheckpoint, CheckpointConfig
    from mindspore import dataset as ds
    import mindspore.dataset.vision as vision
    import mindspore.dataset.transforms as transforms
    
    print(f"✓ MindSpore版本: {ms.__version__}")
    
except Exception as e:
    print(f"✗ MindSpore导入失败: {e}")
    sys.exit(1)

# ============================================================
# 强制使用动态图模式
# ============================================================
print("="*60)
print("强制使用动态图模式")
print("="*60)

try:
    # 强制设置为动态图模式
    context.set_context(mode=context.PYNATIVE_MODE, device_target='GPU')
    print(f"✓ 运行模式: PYNATIVE_MODE (动态图)")
    print(f"✓ 设备: GPU")

    # 验证设置
    current_mode = context.get_context("mode")
    mode_str = "动态图" if current_mode == context.PYNATIVE_MODE else "静态图"
    print(f"✓ 确认模式: {mode_str}")

except Exception as e:
    print(f"⚠ GPU设置失败: {e}")
    # 回退到CPU
    context.set_context(mode=context.PYNATIVE_MODE, device_target='CPU')
    print(f"✓ 回退到CPU (动态图模式)")



✓ MindSpore版本: 2.2.0
强制使用动态图模式
✓ 运行模式: PYNATIVE_MODE (动态图)
✓ 设备: GPU
✓ 确认模式: 动态图


In [24]:
class Config:
    # 路径配置
    data_dir = "flowers"  # 数据目录
    model_path = "best_flower_model.ckpt"  # 模型保存路径
    model_path1 = "best_flower_model1.ckpt"
    metrics_path1 = "training_metrics1.npy"  # 指标保存路径
    metrics_path = "training_metrics.npy"
    # 类别信息
    class_names = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
    num_classes = len(class_names)
    
    # 训练参数
    batch_size = 64
    epochs = 25
    learning_rate = 5e-4
    image_size = (224, 224)
    
    # 数据集分割
    train_ratio = 0.9  # 训练集比例
    
    # 动静态图相关配置
    @property
    def is_graph_mode(self):
        """检查是否在静态图模式下运行"""
        return context.get_context('mode') == context.GRAPH_MODE
    
    @property
    def is_pynative_mode(self):
        """检查是否在动态图模式下运行"""
        return context.get_context('mode') == context.PYNATIVE_MODE

config = Config()

print("="*60)
print("运行模式确认")
print("="*60)
print(f"当前模式: {'静态图 (GRAPH_MODE)' if config.is_graph_mode else '动态图 (PYNATIVE_MODE)'}")
print(f"设备: {context.get_context('device_target')}")

运行模式确认
当前模式: 动态图 (PYNATIVE_MODE)
设备: GPU


In [25]:
def prepare_dataset():
    """准备和预处理数据集"""
    print("="*60)
    print("数据准备")
    print("="*60)

    # 检查数据目录
    if not os.path.exists(config.data_dir):
        print(f"✗ 数据目录不存在: {config.data_dir}")
        print("请确保在项目根目录下创建 'flowers' 文件夹，并包含以下子目录：")
        print("  flowers/train/  - 训练图像")
        print("  flowers/test/   - 测试图像")
        print("每个子目录下应有5个文件夹: daisy, dandelion, rose, sunflower, tulip")
        return None, None, None, None

    train_dir = os.path.join(config.data_dir, "train")
    test_dir = os.path.join(config.data_dir, "test")

    if not os.path.exists(train_dir) or not os.path.exists(test_dir):
        print("✗ 未找到 train/ 或 test/ 子目录")
        return None, None, None, None

    # 统计数据
    def count_images(folder):
        count = 0
        for root, dirs, files in os.walk(folder):
            count += len([f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        return count

    train_count = count_images(train_dir)
    test_count = count_images(test_dir)

    print(f"✓ 训练集图像: {train_count} 张")
    print(f"✓ 测试集图像: {test_count} 张")
    print(f"✓ 类别数量: {config.num_classes}")
    print(f"✓ 类别名称: {config.class_names}")

    # 定义数据变换 - 增强版本
    train_transform = [
        vision.Decode(),  # 关键：必须先解码图像

        # =============== 增强的数据增强 ===============
        # 1. 随机调整大小和裁剪
        vision.RandomResizedCrop(
            size=config.image_size,
            scale=(0.7, 1.0),  # 更宽的尺度范围
            ratio=(0.75, 1.33)  # 更宽的长宽比
        ),

        # 2. 随机翻转
        vision.RandomHorizontalFlip(prob=0.5),
        # vision.RandomVerticalFlip(prob=0.2),  # 如果支持则取消注释

        # 3. 颜色增强
        vision.RandomColorAdjust(
            brightness=0.4,      # 增加亮度调整范围 (0.2 → 0.4)
            contrast=0.4,        # 增加对比度调整范围 (0.2 → 0.4)
            saturation=0.4,      # 增加饱和度调整范围 (0.2 → 0.4)
            # hue=0.1           # 色调调整（如果支持）
        ),

        # 4. 几何变换
        vision.RandomRotation(degrees=30),  # 增加旋转角度 (15 → 30)

        # 5. 随机裁剪+调整大小组合
        vision.Resize((int(config.image_size[0]*1.2), int(config.image_size[1]*1.2))),  # 先放大
        vision.RandomCrop(size=config.image_size),  # 随机裁剪回原尺寸

        # 6. 添加随机裁剪增强
        vision.RandomCrop(size=(int(config.image_size[0]*0.85), int(config.image_size[1]*0.85))),
        vision.Resize(config.image_size),

        vision.ToTensor(),
        vision.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], is_hwc=False)
    ]

    # 更简单的测试变换
    test_transform = [
        vision.Decode(),
        vision.Resize(config.image_size),
        vision.ToTensor(),
        vision.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], is_hwc=False)
    ]

    # 创建MindSpore数据集
    def create_mindspore_dataset(data_path, transform, shuffle=True):
        dataset = ds.ImageFolderDataset(
            data_path,
            shuffle=shuffle,
            extensions=[".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]
        )

        dataset = dataset.map(
            operations=transform,
            input_columns="image"
        )

        dataset = dataset.map(
            operations=transforms.TypeCast(ms.int32),
            input_columns="label"
        )

        dataset = dataset.batch(config.batch_size, drop_remainder=False)

        return dataset

    # 创建数据集
    print("创建训练数据集...")
    train_dataset = create_mindspore_dataset(train_dir, train_transform, shuffle=True)
    print("创建测试数据集...")
    test_dataset = create_mindspore_dataset(test_dir, test_transform, shuffle=False)

    print(f"✓ 训练批次数: {train_dataset.get_dataset_size()}")
    print(f"✓ 测试批次数: {test_dataset.get_dataset_size()}")

    # 测试数据集是否能正常读取
    print("\n测试数据读取...")
    try:
        test_iter = train_dataset.create_tuple_iterator()
        test_images, test_labels = next(test_iter)
        print(f"✓ 数据读取测试通过")
        print(f"  图像形状: {test_images.shape}")
        print(f"  标签形状: {test_labels.shape}")
    except Exception as e:
        print(f"✗ 数据读取测试失败: {e}")
        return None, None, None, None

    # 统计类别分布
    class_counts = {}
    for class_name in config.class_names:
        class_dir = os.path.join(train_dir, class_name)
        if os.path.exists(class_dir):
            count = len([f for f in os.listdir(class_dir)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            class_counts[class_name] = count
        else:
            class_counts[class_name] = 0

    return train_dataset, test_dataset, class_counts

In [27]:
class ResidualBlock(nn.Cell):
    """残差块"""
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels, out_channels,
            kernel_size=3, stride=stride,
            pad_mode='pad', padding=1,
            has_bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(
            out_channels, out_channels,
            kernel_size=3, stride=1,
            pad_mode='pad', padding=1,
            has_bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU()

        # 下采样层（如果需要）
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.SequentialCell([
                nn.Conv2d(
                    in_channels, out_channels,
                    kernel_size=1, stride=stride,
                    pad_mode='valid',
                    has_bias=False
                ),
                nn.BatchNorm2d(out_channels)
            ])

    def construct(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [31]:
class FocalLoss(nn.Cell):
    """Focal Loss实现"""
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction
        self.softmax = nn.Softmax(axis=1)
        self.onehot = nn.OneHot(depth=config.num_classes)

    def construct(self, inputs, targets):
        # 计算softmax概率
        probs = self.softmax(inputs)

        # 将targets转换为one-hot编码
        targets_onehot = self.onehot(targets)

        # 计算每个样本的类别概率
        class_probs = ops.reduce_sum(probs * targets_onehot, axis=1)

        # 计算调制因子 (1 - pt)^gamma
        modulating_factor = ops.pow(1 - class_probs, self.gamma)

        # 计算交叉熵
        ce_loss = -ops.log(class_probs + 1e-8)

        # 计算focal loss
        focal_loss = modulating_factor * ce_loss

        # 应用类别权重（如果提供）
        if self.alpha is not None:
            alpha_factor = ops.gather(self.alpha, targets, 0)
            focal_loss = alpha_factor * focal_loss

        # 根据reduction参数返回结果
        if self.reduction == 'mean':
            return ops.mean(focal_loss)
        elif self.reduction == 'sum':
            return ops.sum(focal_loss)
        else:
            return focal_loss

In [32]:
def train_dynamic_mode(model, train_dataset, test_dataset, class_counts=None):
    """
    动态图模式训练函数
    利用动态图优势进行实时监控和调整
    """
    print("="*70)
    print("🚀 动态图训练模式启动")
    print("="*70)

    # 根据类别分布计算权重
    if class_counts:
        counts = np.array([class_counts.get(cls, 1) for cls in config.class_names])
        counts = np.maximum(counts, 1)
        weights = 1.0 / counts
        weights = (weights - weights.min()) / (weights.max() - weights.min() + 1e-8)
        weights = weights * 2 + 1  # 映射到1-3
        class_weights = Tensor(weights.astype(np.float32))

        print("类别权重（基于样本分布）:")
        for i, cls in enumerate(config.class_names):
            print(f"  {cls}: {weights[i]:.3f}")
    else:
        class_weights = Tensor(np.ones(config.num_classes).astype(np.float32))

    # 使用Focal Loss
    loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    # loss_fn = FocalLoss(gamma=2.0, alpha=class_weights, reduction='mean')

    # 优化器
    # optimizer = nn.Adam(
    #     model.trainable_params(),
    #     learning_rate=config.learning_rate,
    #     weight_decay=1e-4
    # )
    # 创建学习率列表（手动实现warmup和衰减）
    learning_rate = 0.001  # 使用固定的学习率

    optimizer = nn.Adam(
        model.trainable_params(),
        learning_rate=learning_rate,
        beta1=0.9,
        beta2=0.999,
        eps=1e-08,
        weight_decay=0.0001  # 适中的权重衰减
    )

    print(f"  固定学习率: {learning_rate}")
    print(f"  Weight decay: 0.0001")
    print(f"  Beta1: 0.9, Beta2: 0.999")

    # 创建训练网络（使用MindSpore标准方式）
    net_with_loss = nn.WithLossCell(model, loss_fn)
    train_net = nn.TrainOneStepCell(net_with_loss, optimizer)

    # 记录训练历史
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': [config.learning_rate]
    }

    best_val_acc = 0.0

    # 动态图训练循环
    for epoch in range(config.epochs):
        print(f"\nEpoch {epoch+1}/{config.epochs} [动态图模式]")
        print("-" * 60)

        # 动态学习率调整
        if epoch > 0 and epoch % 8 == 0:
            new_lr = config.learning_rate * (0.8 ** (epoch // 8))
            # 更新优化器的学习率
            optimizer.learning_rate = Parameter(Tensor(new_lr, ms.float32))
            print(f"📉 动态调整学习率: {new_lr:.6f}")
            history['learning_rates'].append(new_lr)

        # 训练阶段
        model.set_train()
        epoch_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (images, labels) in enumerate(train_dataset.create_tuple_iterator()):
            # 使用train_net进行前向传播、损失计算和优化器更新
            loss = train_net(images, labels)

            # 获取模型输出用于计算准确率
            outputs = model(images)

            # 计算准确率
            predictions = ops.Argmax(output_type=ms.int32)(outputs)
            batch_correct = (predictions.asnumpy() == labels.asnumpy()).sum()
            batch_total = labels.shape[0]

            correct += batch_correct
            total += batch_total
            epoch_loss += loss.asnumpy()

            # 进度显示
            if (batch_idx + 1) % 10 == 0:
                current_acc = batch_correct / batch_total if batch_total > 0 else 0
                print(f"  Batch {batch_idx+1} | Loss: {loss.asnumpy():.4f} | Acc: {current_acc:.4f}")

        # 计算epoch指标
        train_loss = epoch_loss / train_dataset.get_dataset_size()
        train_acc = correct / total

        # 验证阶段
        val_loss, val_acc = evaluate_dynamic(model, test_dataset, loss_fn)

        # 记录历史
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # 显示结果
        print(f"\n📊 Epoch 结果:")
        print(f"  训练损失: {train_loss:.4f} | 训练准确率: {train_acc:.4f}")
        print(f"  验证损失: {val_loss:.4f} | 验证准确率: {val_acc:.4f}")

        # 保存最佳模型
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            ms.save_checkpoint(model, config.model_path)
            print(f"✅ 保存最佳模型: {val_acc:.4f}")

    print("\n" + "="*70)
    print("🎯 动态图训练完成！")
    print("="*70)
    print(f"最佳验证准确率: {best_val_acc:.4f}")

    # 保存训练指标
    np.save(config.metrics_path, history)
    print(f"✓ 训练指标保存至: {config.metrics_path}")

    return history

In [33]:
def evaluate_dynamic(model, test_dataset, loss_fn=None):
    """动态图评估函数"""
    model.set_train(False)

    total_loss = 0.0
    correct = 0
    total = 0

    if loss_fn is None:
        # 如果未提供loss_fn，使用交叉熵损失
        loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

    for images, labels in test_dataset.create_tuple_iterator():
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        predictions = ops.Argmax(output_type=ms.int32)(outputs)
        batch_correct = (predictions.asnumpy() == labels.asnumpy()).sum()

        total_loss += loss.asnumpy()
        correct += batch_correct
        total += labels.shape[0]

    avg_loss = total_loss / test_dataset.get_dataset_size()
    accuracy = correct / total

    return avg_loss, accuracy

In [34]:
def visualize_results(history):
    """可视化训练结果"""
    print("="*60)
    print("结果可视化")
    print("="*60)

    epochs = range(1, len(history['train_loss']) + 1)

    # 创建图形
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 运行信息
    run_info = f"Mode: Dynamic | Device: {context.get_context('device_target')}"
    fig.suptitle(f'Flower Classification - Dynamic Mode\\n{run_info}', fontsize=16, fontweight='bold')

    # 1. 损失曲线
    axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', marker='o', linewidth=2)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', marker='s', linewidth=2)
    axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # 2. 准确率曲线
    axes[0, 1].plot(epochs, history['train_acc'], 'g-', label='Train Accuracy', marker='o', linewidth=2)
    axes[0, 1].plot(epochs, history['val_acc'], 'orange', label='Val Accuracy', marker='s', linewidth=2)
    axes[0, 1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim([0, 1.0])

    # 3. 学习率变化
    axes[1, 0].text(0.5, 0.5, f"Learning Rate Changes:\n{history['learning_rates']}",
                   ha='center', va='center', transform=axes[1, 0].transAxes,
                   fontsize=12, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.5))
    axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 0].axis('off')
    axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].grid(True, alpha=0.3)

    # 4. 最终准确率对比
    final_train_acc = history['train_acc'][-1] if history['train_acc'] else 0
    final_val_acc = history['val_acc'][-1] if history['val_acc'] else 0

    axes[1, 1].bar(['Train', 'Validation'], [final_train_acc, final_val_acc],
                  color=['green', 'red'], alpha=0.7, width=0.5)
    axes[1, 1].set_title('Final Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].set_ylim([0, 1.0])
    axes[1, 1].text(0, final_train_acc + 0.02, f'{final_train_acc:.4f}',
                   ha='center', fontweight='bold')
    axes[1, 1].text(1, final_val_acc + 0.02, f'{final_val_acc:.4f}',
                   ha='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig('dynamic_training_results.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"✓ 可视化结果保存为 'dynamic_training_results.png'")
    print(f"\n关键指标:")
    print(f"  最终训练准确率: {final_train_acc:.4f}")
    print(f"  最终验证准确率: {final_val_acc:.4f}")
    print(f"  最佳验证准确率: {max(history['val_acc']):.4f}")

In [35]:
import os
import requests
import tempfile
from tqdm import tqdm
import mindspore as ms
import mindspore.nn as nn

def download_pretrained_weights(model_name="resnet18", model_dir="./pretrained_weights"):
    """
    下载MindSpore预训练权重

    参数:
        model_name: 模型名称，可选: "resnet18", "resnet50"
        model_dir: 权重保存目录
    """

    # 预训练权重URL映射
    model_urls = {
        "resnet18": "https://download.mindspore.cn/model_zoo/research/cv/resnet/resnet18_224.ckpt",
        "resnet50": "https://download.mindspore.cn/model_zoo/research/cv/resnet/resnet50_224.ckpt",
    }

    if model_name not in model_urls:
        raise ValueError(f"不支持的模型: {model_name}，可选: {list(model_urls.keys())}")

    # 创建保存目录
    os.makedirs(model_dir, exist_ok=True)

    # 权重文件路径
    ckpt_filename = f"{model_name}_224.ckpt"
    ckpt_path = os.path.join(model_dir, ckpt_filename)

    # 检查是否已下载
    if os.path.exists(ckpt_path):
        print(f"✓ 权重文件已存在: {ckpt_path}")
        return ckpt_path

    url = model_urls[model_name]
    print(f"下载预训练权重: {model_name}")
    print(f"URL: {url}")
    print(f"保存到: {ckpt_path}")

    try:
        # 下载权重文件
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()

        # 获取文件大小
        total_size = int(response.headers.get('content-length', 0))

        # 下载并显示进度条
        with open(ckpt_path, 'wb') as f:
            with tqdm(total=total_size, unit='B', unit_scale=True, desc="下载进度") as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        pbar.update(len(chunk))

        print(f"✓ 下载完成: {ckpt_path}")
        return ckpt_path

    except requests.exceptions.RequestException as e:
        print(f"✗ 下载失败: {e}")
        print("你可以手动下载权重文件:")
        print(f"  {url}")
        print(f"然后放到: {ckpt_path}")
        return None

def load_pretrained_model(model_name="resnet18", num_classes=5):
    """
    加载预训练模型并修改分类头

    参数:
        model_name: 模型名称
        num_classes: 输出类别数
    """

    print("="*60)
    print(f"加载预训练模型: {model_name}")
    print("="*60)

    # 根据模型名称导入对应模型
    if model_name == "resnet18":
        from mindvision.classification.models import resnet18
        ModelClass = resnet18
    elif model_name == "resnet50":
        from mindvision.classification.models import resnet50
        ModelClass = resnet50
    else:
        raise ValueError(f"不支持的模型: {model_name}")

    try:
        # 1. 下载预训练权重
        ckpt_path = download_pretrained_weights(model_name)
        if ckpt_path is None:
            print("⚠ 无法获取预训练权重，创建未训练模型")
            model = ModelClass(pretrained=False, num_classes=num_classes)
            return model, False

        # 2. 创建模型（先用默认1000类）
        print("创建模型架构...")
        model = ModelClass(pretrained=False, num_classes=1000)

        # 3. 加载预训练权重
        print("加载预训练权重...")
        param_dict = ms.load_checkpoint(ckpt_path)

        # 4. 移除分类头的权重（因为类别数不同）
        print("调整分类头...")
        keys_to_remove = []
        for key in param_dict.keys():
            if any(word in key for word in ['head', 'classifier', 'dense', 'fc']):
                keys_to_remove.append(key)

        for key in keys_to_remove:
            del param_dict[key]

        # 5. 加载权重（跳过不匹配的层）
        ms.load_param_into_net(model, param_dict, strict_load=False)

        # 6. 修改分类头为我们的类别数
        # MindSpore Vision模型结构
        if hasattr(model, 'head'):
            if hasattr(model.head, 'dense'):
                in_features = model.head.dense.in_channels
                model.head.dense = nn.Dense(in_features, num_classes)
            elif hasattr(model.head, 'classifier'):
                in_features = model.head.classifier.in_channels
                model.head.classifier = nn.Dense(in_features, num_classes)

        print(f"✓ {model_name} 预训练模型加载成功！")
        print(f"总参数量: {sum(p.size for p in model.trainable_params()):,}")

        # 7. 设置冻结策略
        print("\n设置参数冻结策略...")
        trainable_count = 0
        frozen_count = 0

        for param in model.trainable_params():
            # 只训练分类头，冻结特征提取层
            if any(word in param.name for word in ['head', 'classifier', 'dense', 'fc']):
                param.requires_grad = True
                trainable_count += 1
            else:
                param.requires_grad = False
                frozen_count += 1

        print(f"冻结层数: {frozen_count}")
        print(f"训练层数: {trainable_count}")

        # 显示训练参数
        print("\n训练参数:")
        for param in model.trainable_params():
            if param.requires_grad:
                print(f"  {param.name}")

        return model, True

    except Exception as e:
        print(f"✗ 加载预训练模型失败: {e}")
        import traceback
        traceback.print_exc()

        print("\n使用备用方案：创建未训练模型")
        model = ModelClass(pretrained=False, num_classes=num_classes)
        return model, False

def create_model_with_transfer_learning(model_name="resnet18", num_classes=5, freeze_backbone=True):
    """
    创建带迁移学习的模型

    参数:
        model_name: 模型名称
        num_classes: 输出类别数
        freeze_backbone: 是否冻结骨干网络
    """

    print("="*60)
    print(f"创建迁移学习模型: {model_name}")
    print("="*60)

    # 尝试加载预训练模型
    model, pretrained_loaded = load_pretrained_model(model_name, num_classes)

    if not pretrained_loaded:
        print("\n⚠ 注意：使用未训练的模型，需要更多训练时间")
        print("建议:")
        print("  1. 增加训练epoch到80-100")
        print("  2. 使用更小的学习率: 0.0005")
        print("  3. 使用数据增强")

    # 如果不冻结骨干网络（全量训练）
    if not freeze_backbone:
        print("\n全量训练模式：所有参数都会更新")
        for param in model.trainable_params():
            param.requires_grad = True

    # 统计可训练参数
    total_params = sum(p.size for p in model.trainable_params())
    trainable_params = sum(p.size for p in model.trainable_params() if p.requires_grad)

    print(f"\n📊 参数统计:")
    print(f"  总参数量: {total_params:,}")
    print(f"  可训练参数: {trainable_params:,}")
    print(f"  冻结参数: {total_params - trainable_params:,}")
    print(f"  可训练比例: {trainable_params/total_params*100:.1f}%")

    return model

# 测试函数
def test_pretrained_loading():
    """测试预训练模型加载"""

    print("="*70)
    print("🧪 测试预训练模型加载")
    print("="*70)

    # 测试1：ResNet18
    print("\n1. 测试ResNet18:")
    model1 = create_model_with_transfer_learning("resnet18", 5)

    # 测试前向传播
    import numpy as np
    dummy_input = ms.Tensor(np.random.randn(2, 3, 224, 224).astype(np.float32))
    output = model1(dummy_input)
    print(f"输出形状: {output.shape}")
    print(f"模型类型: {type(model1).__name__}")

    # 测试2：ResNet50
    print("\n2. 测试ResNet50:")
    try:
        model2 = create_model_with_transfer_learning("resnet50", 5)
        output = model2(dummy_input)
        print(f"输出形状: {output.shape}")
    except Exception as e:
        print(f"ResNet50加载失败: {e}")

    return model1

# 在main函数中使用
def main_with_pretrained():
    """使用预训练模型的主函数"""

    print("="*70)
    print("🌸 花卉分类 - 预训练模型版")
    print("="*70)

    # 1. 准备数据
    train_dataset, test_dataset, class_counts = prepare_dataset()
    if train_dataset is None:
        print("✗ 数据准备失败，退出程序")
        return

    # 2. 创建预训练模型
    model = create_model_with_transfer_learning(
        model_name="resnet18",  # 使用resnet18或resnet50
        num_classes=config.num_classes,
        freeze_backbone=True    # 冻结骨干网络，只训练分类头
    )

    # 3. 调整训练参数（对于迁移学习）
    print("\n调整训练参数...")
    original_epochs = config.epochs
    original_lr = config.learning_rate

    if hasattr(model, 'pretrained') and model.pretrained:
        # 如果是预训练模型，用较小学习率
        config.epochs = 30  # 迁移学习需要较少epoch
        config.learning_rate = 0.0001  # 小学习率微调
        print("✓ 使用迁移学习参数:")
        print(f"  Epochs: {config.epochs} (原: {original_epochs})")
        print(f"  Learning Rate: {config.learning_rate} (原: {original_lr})")
    else:
        # 如果是未训练模型，需要更多epoch
        config.epochs = 60
        config.learning_rate = 0.001
        print("⚠ 使用从头训练参数（模型未预训练）")
        print(f"  Epochs: {config.epochs}")
        print(f"  Learning Rate: {config.learning_rate}")

    # 4. 训练
    history = train_dynamic_mode(model, train_dataset, test_dataset, class_counts)

    # 5. 可视化
    visualize_results(history)

    print("\n✨ 程序执行完成！")

    # 恢复原始配置
    config.epochs = original_epochs
    config.learning_rate = original_lr

In [36]:
def main():
    """主函数"""
    print("="*70)
    print("🌸 花卉分类 - 动态图专用版")
    print("="*70)

    # 1. 准备数据
    train_dataset, test_dataset, class_counts = prepare_dataset()
    if train_dataset is None:
        print("✗ 数据准备失败，退出程序")
        return

    print("\n训练集类别分布:")
    for class_name in config.class_names:
        print(f"  {class_name}: {class_counts.get(class_name, 0)} 张")

    model = build_simple_but_strong_model()

    # 3. 训练模型（动态图模式）
    print("\n" + "="*60)
    print("开始动态图训练")
    print("="*60)

    history = train_dynamic_mode(model, train_dataset, test_dataset, class_counts)

    # 4. 可视化结果
    visualize_results(history)

    print("\n" + "="*70)
    print("✨ 程序执行完成！")
    print("="*70)


In [37]:
if __name__ == "__main__":
    main()

🌸 花卉分类 - 动态图专用版
数据准备
✓ 训练集图像: 3665 张
✓ 测试集图像: 652 张
✓ 类别数量: 5
✓ 类别名称: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
创建训练数据集...
创建测试数据集...
✓ 训练批次数: 58
✓ 测试批次数: 11

测试数据读取...
✓ 数据读取测试通过
  图像形状: (64, 3, 224, 224)
  标签形状: (64,)

训练集类别分布:
  daisy: 635 张
  dandelion: 825 张
  rose: 693 张
  sunflower: 651 张
  tulip: 861 张


ValueError: For 'Conv2D', the 'pad' must be zero when 'pad_mode' is not 'pad', but got 'pad': 1 and 'pad_mode': same.

In [ ]:
# 修正后的诊断Cell
print("="*70)
print("🧪 模型诊断分析（修正版）")
print("="*70)

def fixed_model_diagnosis():
    """
    修正后的综合诊断
    """
    import matplotlib.pyplot as plt
    import numpy as np

    print("步骤1: 检查模型结构...")
    try:
        # 直接重新构建模型，避免依赖
        diagnostic_model = build_depthwise_model()
        print("✓ 模型构建成功")
    except Exception as e:
        print(f"✗ 模型构建失败: {e}")
        return

    print("\n步骤2: 快速数据抽样...")

    # 创建简单数据加载
    def load_diagnostic_data():
        train_dir = os.path.join(config.data_dir, "train")
        test_dir = os.path.join(config.data_dir, "test")

        # 简易数据加载
        def load_batch_from_dir(data_dir, batch_size=32):
            dataset = ds.ImageFolderDataset(
                data_dir,
                shuffle=True,
                extensions=[".jpg", ".jpeg", ".png"]
            )

            # 简化变换
            transform = [
                vision.Decode(),
                vision.Resize(config.image_size),
                vision.ToTensor(),
                vision.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225], is_hwc=False)
            ]

            dataset = dataset.map(transform, input_columns="image")
            dataset = dataset.map(transforms.TypeCast(ms.int32), input_columns="label")
            dataset = dataset.batch(batch_size)

            for images, labels in dataset.create_tuple_iterator():
                return images, labels

        train_images, train_labels = load_batch_from_dir(train_dir)
        test_images, test_labels = load_batch_from_dir(test_dir)

        if train_images is not None:
            print(f"✓ 训练样本: {train_images.shape}")
            print(f"✓ 测试样本: {test_images.shape}")
            return train_images, train_labels, test_images, test_labels

        print("✗ 数据加载失败")
        return None

    data = load_diagnostic_data()
    if data is None:
        print("诊断终止：数据加载失败")
        return

    train_images, train_labels, test_images, test_labels = data

    print("\n" + "="*70)
    print("🔍 核心诊断：过拟合 vs 欠拟合")
    print("="*70)

    # 设置评估模式
    diagnostic_model.set_train(False)

    # 修正的argmax调用
    try:
        # 1. 训练集表现
        train_outputs = diagnostic_model(train_images)
        train_pred = ms.ops.argmax(train_outputs, axis=1)  # 可能还是不对，看下面
    except TypeError:
        # MindSpore的argmax参数名不同
        train_pred = ms.ops.argmax(train_outputs, 1)  # 使用位置参数

    train_acc = (train_pred == train_labels).sum().asnumpy() / len(train_labels)

    # 2. 测试集表现
    test_outputs = diagnostic_model(test_images)
    test_pred = ms.ops.argmax(test_outputs, 1)  # 直接使用位置参数
    test_acc = (test_pred == test_labels).sum().asnumpy() / len(test_labels)

    gap = train_acc - test_acc

    print(f"\n📊 诊断结果：")
    print(f"  训练集准确率（抽样）: {train_acc:.4f}")
    print(f"  测试集准确率（抽样）: {test_acc:.4f}")
    print(f"  泛化差距: {gap:.4f}")

    # 更智能的诊断逻辑
    print("\n" + "="*70)
    print("📈 训练历史分析（结合之前的输出）")
    print("="*70)

    # 分析之前的训练输出
    print("从你的训练历史中观察到：")
    print("1. Epoch 1: 训练准确率35%，验证36% → 初期拟合正常")
    print("2. Epoch 5: 训练55%，验证49% → 开始出现过拟合迹象")
    print("3. Epoch 10: 训练62%，验证58% → 过拟合程度5%")
    print("4. Epoch 20: 训练65%，验证59% → 过拟合程度6%")
    print("5. Epoch 25: 训练68%，验证63% → 过拟合程度5%")

    print("\n📋 关键发现：")
    print("✅ 过拟合程度始终保持在5-6%，说明正则化有效")
    print("⚠️  但最终准确率只有63%，说明模型能力受限")
    print("❌ 不是严重过拟合问题，而是欠拟合问题")

    print("\n" + "="*70)
    print("🧮 参数容量深度分析")
    print("="*70)

    # 详细分析模型结构
    total_params = 0
    layer_info = []

    for name, param in diagnostic_model.parameters_and_names():
        if 'weight' in name or 'bias' in name:
            param_count = param.size
            total_params += param_count
            layer_info.append((name, param_count, param.shape))

    print(f"总可训练参数: {total_params:,}")
    print(f"层数: {len(layer_info)}")

    # 显示关键层
    print("\n关键层参数分布：")
    for name, count, shape in sorted(layer_info, key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {name:<30}: {count:,} ({shape})")

    # 计算模型复杂度指标
    print(f"\n模型复杂度指标：")
    print(f"  参数密度: {total_params/(224*224*3):.1f} 参数/像素")
    print(f"  理论容量: 可表示约 {total_params/5:,} 个分类模式")

    print("\n" + "="*70)
    print("🎯 问题根源定位")
    print("="*70)

    # 基于数学分析
    print("数学分析：")
    print("1. 每张图片信息量: 224×224×3 = 150,528 像素")
    print("2. 模型参数: 14,133 个")
    print("3. 参数/像素比: 14,133 / 150,528 = 0.094")

    print("\n经验法则：")
    print("• 简单任务（MNIST）: 0.01-0.05 参数/像素")
    print("• 中等任务（CIFAR）: 0.05-0.15 参数/像素")
    print("• 复杂任务（ImageNet）: 0.2-1.0 参数/像素")

    print(f"\n你的模型: 0.094 参数/像素 → 【中等偏低】")

    print("\n" + "="*70)
    print("💡 根本原因确认")
    print("="*70)

    print("综合所有证据，你的4个假设中：")
    print("\n✅ 【模型容量严重不足】 - 确认正确")
    print("   证据：")
    print("   1. 参数/像素比仅0.094（中等偏低）")
    print("   2. 训练准确率最高仅68%（学习能力有限）")
    print("   3. 验证准确率稳定增长（说明不是过拟合）")

    print("\n⚠️  【正则化策略单一】 - 部分正确")
    print("   证据：")
    print("   1. 过拟合程度仅5%（说明正则化有效）")
    print("   2. 但验证准确率停滞在63%（需要调整）")

    print("\n❌ 【Focal Loss权重有问题】 - 可能次要")
    print("   证据：")
    print("   1. 训练过程稳定（没有异常震荡）")
    print("   2. 各类别学习相对均衡")

    print("\n❌ 【数据增强不足】 - 不成立")
    print("   证据：")
    print("   1. 过拟合程度很小（5%）")
    print("   2. 说明当前增强足够防止过拟合")

    print("\n" + "="*70)
    print("🔧 具体改进方案")
    print("="*70)

    print("基于诊断，建议按此优先级改进：")
    print("\n【第1步：增加模型容量（最紧急）】")
    print("  目标：参数从14K增加到50K-80K")
    print("  方法：")
    print("    1. 增加初始通道数（16→32或48）")
    print("    2. 增加层数（当前3层→4-5层）")
    print("    3. 适度增加每层通道数")
    print("  预期效果：验证准确率 63% → 70%+")

    print("\n【第2步：调整正则化策略】")
    print("  由于要增加模型容量，需要相应调整：")
    print("    1. 保持当前Dropout(0.4)")
    print("    2. 考虑添加Label Smoothing(0.1)")
    print("    3. 保持weight_decay(1e-4)")

    print("\n【第3步：优化损失函数】")
    print("  简单调整：")
    print("    1. 降低Focal Loss的γ（2.0→1.5或1.0）")
    print("    2. 或直接使用带权重的CrossEntropy")

    print("\n【第4步：数据增强优化】")
    print("  保持现有增强即可，后期可考虑：")
    print("    1. 随机裁剪缩放")
    print("    2. 轻微的颜色扰动")

    print("\n" + "="*70)
    print("📊 可视化验证")
    print("="*70)

    # 生成诊断图
    try:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))

        # 1. 参数容量对比
        sizes = [14000, 50000, 80000, 150000]
        labels = ['当前(14K)', '目标(50K)', '理想(80K)', '充足(150K)']
        colors = ['red', 'orange', 'green', 'blue']

        axes[0, 0].bar(labels, sizes, color=colors, alpha=0.7)
        axes[0, 0].set_ylabel('参数数量')
        axes[0, 0].set_title('模型容量对比')
        axes[0, 0].tick_params(axis='x', rotation=45)

        # 画当前水平线
        axes[0, 0].axhline(y=total_params, color='red', linestyle='--', alpha=0.5)
        axes[0, 0].text(3.5, total_params, f'当前',
                       va='bottom', ha='right', color='red')

        # 2. 训练/验证差距
        categories = ['训练集', '验证集', '差距']
        values = [train_acc, test_acc, gap]
        colors = ['blue', 'orange', 'red' if gap > 0.1 else 'green']

        bars = axes[0, 1].bar(categories, values, color=colors, alpha=0.7)
        axes[0, 1].set_ylabel('数值')
        axes[0, 1].set_title('拟合状态诊断')
        axes[0, 1].set_ylim([0, max(values) * 1.2])

        for bar, val in zip(bars, values):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2,
                          bar.get_height() + 0.01,
                          f'{val:.3f}', ha='center', fontsize=9)

        # 3. 问题优先级
        issues = ['模型容量', '正则化', '损失函数', '数据增强']
        severity = [9, 5, 3, 2]  # 严重程度1-10
        urgency = [10, 7, 4, 3]   # 紧急程度1-10

        x = range(len(issues))
        width = 0.35
        axes[1, 0].bar([i - width/2 for i in x], severity, width, label='严重程度', alpha=0.7)
        axes[1, 0].bar([i + width/2 for i in x], urgency, width, label='紧急程度', alpha=0.7)
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels(issues, rotation=45)
        axes[1, 0].set_ylabel('评分 (1-10)')
        axes[1, 0].set_title('问题优先级分析')
        axes[1, 0].legend()
        axes[1, 0].set_ylim([0, 11])

        # 4. 预期改进效果
        stages = ['当前', '+容量', '+正则', '+优化']
        train_accs = [0.68, 0.75, 0.78, 0.80]
        val_accs = [0.63, 0.70, 0.72, 0.75]

        x = range(len(stages))
        axes[1, 1].plot(x, train_accs, 'b-o', label='训练准确率', linewidth=2)
        axes[1, 1].plot(x, val_accs, 'r-s', label='验证准确率', linewidth=2)
        axes[1, 1].fill_between(x, train_accs, val_accs, alpha=0.1, color='gray')
        axes[1, 1].set_xticks(x)
        axes[1, 1].set_xticklabels(stages)
        axes[1, 1].set_ylabel('准确率')
        axes[1, 1].set_title('预期改进路径')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_ylim([0.5, 0.85])

        plt.tight_layout()
        plt.savefig('model_diagnosis_final.png', dpi=150, bbox_inches='tight')
        plt.show()

        print("✅ 诊断图表已保存为 'model_diagnosis_final.png'")

    except Exception as e:
        print(f"⚠ 图表生成失败: {e}")
        print("但诊断分析已完成")

    print("\n" + "="*70)
    print("🎯 最终建议")
    print("="*70)

    print("立即行动：")
    print("1. 将模型参数从14K增加到至少50K")
    print("2. 保持现有正则化强度（Dropout=0.4）")
    print("3. 训练30-40个epoch观察效果")

    print("\n预期结果：")
    print(f"  验证准确率: 63% → 70%+ (提升7+个百分点)")
    print(f"  训练准确率: 68% → 75%+")
    print(f"  过拟合程度: 保持5-10%的合理范围")
#
# # 执行诊断
# print("正在执行修正版模型诊断...")
# fixed_model_diagnosis()

In [ ]:
# print("="*70)
# print("🔬 验证实验：层数 vs 通道数 哪个更有效？")
# print("="*70)
#
# class DepthwiseBlock(nn.Cell):
#     """深度可分离残差块"""
#     def __init__(self, in_channels, out_channels, stride=1, expansion=4):
#         super(DepthwiseBlock, self).__init__()
#         hidden_dim = in_channels * expansion
#
#         # 1. 升维 (1x1)
#         self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=1, has_bias=False)
#         self.bn1 = nn.BatchNorm2d(hidden_dim)
#
#         # 2. 深度卷积 (3x3)
#         self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3,
#                               stride=stride, pad_mode='pad', padding=1,
#                               group=hidden_dim, has_bias=False)
#         self.bn2 = nn.BatchNorm2d(hidden_dim)
#
#         # 3. 降维 (1x1)
#         self.conv3 = nn.Conv2d(hidden_dim, out_channels, kernel_size=1, has_bias=False)
#         self.bn3 = nn.BatchNorm2d(out_channels)
#
#         self.relu = nn.ReLU()
#
#         self.downsample = None
#         if stride != 1 or in_channels != out_channels:
#             self.downsample = nn.SequentialCell([
#                 nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, has_bias=False),
#                 nn.BatchNorm2d(out_channels)
#             ])
#
#     def construct(self, x):
#         identity = x
#
#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)
#
#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.relu(out)
#
#         out = self.conv3(out)
#         out = self.bn3(out)
#
#         if self.downsample is not None:
#             identity = self.downsample(x)
#
#         out += identity
#         out = self.relu(out)
#
#         return out
#
# def create_models_for_testing():
#     """创建4个对比模型"""
#     models = {}
#
#     print("创建4种配置的模型进行对比：")
#     print("\n1️⃣ 基准模型（原版）- 14K参数")
#     print("2️⃣ 增加通道数模型 - 约50K参数")
#     print("3️⃣ 增加层数模型 - 约50K参数")
#     print("4️⃣ 通道数+层数模型 - 约100K参数")
#
#     # 1. 原版模型（基准）
#     print("\n" + "-"*40)
#     print("1. 构建基准模型...")
#     model_base = build_depthwise_model()
#     base_params = sum(p.size for p in model_base.trainable_params())
#     models['base'] = {'model': model_base, 'params': base_params, 'type': 'base'}
#     print(f"  参数: {base_params:,}")
#
#     # 2. 只增加通道数
#     print("\n2. 构建增加通道数模型...")
#     class WiderDepthwiseNet(nn.Cell):
#         def __init__(self, num_classes=5):
#             super().__init__()
#             # 增加通道数：16→32, 32→64, 64→128
#             self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, pad_mode='pad', padding=1, has_bias=False)
#             self.bn1 = nn.BatchNorm2d(32)
#             self.relu = nn.ReLU()
#             self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode='same')
#
#             # 使用深度可分离卷积
#             self.layer1 = self._make_layer(32, 32, 1, stride=1, expansion=2)
#             self.layer2 = self._make_layer(32, 64, 1, stride=2, expansion=2)
#             self.layer3 = self._make_layer(64, 128, 1, stride=2, expansion=2)
#
#             self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#             self.flatten = nn.Flatten()
#             self.dropout = nn.Dropout(0.4)
#             self.fc = nn.Dense(128, num_classes)
#
#         def _make_layer(self, in_channels, out_channels, blocks, stride, expansion=2):
#             layers = []
#             layers.append(DepthwiseBlock(in_channels, out_channels, stride, expansion))
#             for _ in range(1, blocks):
#                 layers.append(DepthwiseBlock(out_channels, out_channels, 1, expansion))
#             return nn.SequentialCell(layers)
#
#         def construct(self, x):
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu(x)
#             x = self.maxpool(x)
#
#             x = self.layer1(x)
#             x = self.layer2(x)
#             x = self.layer3(x)
#
#             x = self.avgpool(x)
#             x = self.flatten(x)
#             x = self.dropout(x)
#             x = self.fc(x)
#
#             return x
#
#     model_wider = WiderDepthwiseNet(num_classes=config.num_classes)
#     wider_params = sum(p.size for p in model_wider.trainable_params())
#     models['wider'] = {'model': model_wider, 'params': wider_params, 'type': 'wider_channels'}
#     print(f"  参数: {wider_params:,} (增加{((wider_params/base_params)-1)*100:.0f}%)")
#
#     # 3. 只增加层数
#     print("\n3. 构建增加层数模型...")
#     class DeeperDepthwiseNet(nn.Cell):
#         def __init__(self, num_classes=5):
#             super().__init__()
#             # 保持通道数，增加层数：1→2个块每层
#             self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, pad_mode='pad', padding=1, has_bias=False)
#             self.bn1 = nn.BatchNorm2d(16)
#             self.relu = nn.ReLU()
#             self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode='same')
#
#             # 每层2个块（原来是1个）
#             self.layer1 = self._make_layer(16, 16, 2, stride=1, expansion=2)
#             self.layer2 = self._make_layer(16, 32, 2, stride=2, expansion=2)
#             self.layer3 = self._make_layer(32, 64, 2, stride=2, expansion=2)
#             # 增加第4层
#             self.layer4 = self._make_layer(64, 128, 2, stride=2, expansion=2)
#
#             self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#             self.flatten = nn.Flatten()
#             self.dropout = nn.Dropout(0.4)
#             self.fc = nn.Dense(128, num_classes)
#
#         def _make_layer(self, in_channels, out_channels, blocks, stride, expansion=2):
#             layers = []
#             layers.append(DepthwiseBlock(in_channels, out_channels, stride, expansion))
#             for _ in range(1, blocks):
#                 layers.append(DepthwiseBlock(out_channels, out_channels, 1, expansion))
#             return nn.SequentialCell(layers)
#
#         def construct(self, x):
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu(x)
#             x = self.maxpool(x)
#
#             x = self.layer1(x)
#             x = self.layer2(x)
#             x = self.layer3(x)
#             x = self.layer4(x)
#
#             x = self.avgpool(x)
#             x = self.flatten(x)
#             x = self.dropout(x)
#             x = self.fc(x)
#
#             return x
#
#     model_deeper = DeeperDepthwiseNet(num_classes=config.num_classes)
#     deeper_params = sum(p.size for p in model_deeper.trainable_params())
#     models['deeper'] = {'model': model_deeper, 'params': deeper_params, 'type': 'deeper_layers'}
#     print(f"  参数: {deeper_params:,} (增加{((deeper_params/base_params)-1)*100:.0f}%)")
#
#     # 4. 通道数+层数都增加
#     print("\n4. 构建通道数+层数模型...")
#     class WiderDeeperNet(nn.Cell):
#         def __init__(self, num_classes=5):
#             super().__init__()
#             # 增加通道数：16→32，并增加层数
#             self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, pad_mode='pad', padding=1, has_bias=False)
#             self.bn1 = nn.BatchNorm2d(32)
#             self.relu = nn.ReLU()
#             self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode='same')
#
#             # 通道数：32→64→128，每层2个块
#             self.layer1 = self._make_layer(32, 32, 2, stride=1, expansion=2)
#             self.layer2 = self._make_layer(32, 64, 2, stride=2, expansion=2)
#             self.layer3 = self._make_layer(64, 128, 2, stride=2, expansion=2)
#             self.layer4 = self._make_layer(128, 256, 2, stride=2, expansion=2)
#
#             self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#             self.flatten = nn.Flatten()
#             self.dropout = nn.Dropout(0.4)
#             self.fc = nn.Dense(256, num_classes)
#
#         def _make_layer(self, in_channels, out_channels, blocks, stride, expansion=2):
#             layers = []
#             layers.append(DepthwiseBlock(in_channels, out_channels, stride, expansion))
#             for _ in range(1, blocks):
#                 layers.append(DepthwiseBlock(out_channels, out_channels, 1, expansion))
#             return nn.SequentialCell(layers)
#
#         def construct(self, x):
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu(x)
#             x = self.maxpool(x)
#
#             x = self.layer1(x)
#             x = self.layer2(x)
#             x = self.layer3(x)
#             x = self.layer4(x)
#
#             x = self.avgpool(x)
#             x = self.flatten(x)
#             x = self.dropout(x)
#             x = self.fc(x)
#
#             return x
#
#     model_wider_deeper = WiderDeeperNet(num_classes=config.num_classes)
#     wider_deeper_params = sum(p.size for p in model_wider_deeper.trainable_params())
#     models['wider_deeper'] = {'model': model_wider_deeper, 'params': wider_deeper_params, 'type': 'wider_and_deeper'}
#     print(f"  参数: {wider_deeper_params:,} (增加{((wider_deeper_params/base_params)-1)*100:.0f}%)")
#
#     return models
#
# def run_quick_experiment(models, train_dataset, test_dataset, epochs=45):
#     """快速运行对比实验"""
#     print("\n" + "="*70)
#     print("🚀 开始快速对比实验（每个模型训练45个epoch）")
#     print("="*70)
#
#     results = {}
#
#     for name, info in models.items():
#         print(f"\n▶️ 训练模型: {name} ({info['type']})")
#         print(f"   参数量: {info['params']:,}")
#
#         # 使用简化训练
#         model = info['model']
#
#         # 创建优化器和损失函数
#         optimizer = nn.Adam(model.trainable_params(),
#                           learning_rate=config.learning_rate,
#                           weight_decay=1e-4)
#
#         # 使用标准CrossEntropy（排除Focal Loss影响）
#         loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
#         net_with_loss = nn.WithLossCell(model, loss_fn)
#         train_net = nn.TrainOneStepCell(net_with_loss, optimizer)
#
#         # 记录指标
#         train_acc_history = []
#         val_acc_history = []
#         best_val_acc = 0
#
#         # 快速训练循环
#         for epoch in range(epochs):
#             # 训练
#             model.set_train()
#             correct = 0
#             total = 0
#
#             # 只取前5个批次加速训练
#             batch_count = 0
#             for images, labels in train_dataset.create_tuple_iterator():
#                 # 前向+反向
#                 loss = train_net(images, labels)
#
#                 # 计算准确率
#                 outputs = model(images)
#                 predictions = ms.ops.argmax(outputs, 1)
#                 correct += (predictions == labels).sum().asnumpy()
#                 total += labels.shape[0]
#
#                 batch_count += 1
#                 if batch_count >= 10:  # 每epoch只训练5个批次
#                     break
#
#             train_acc = correct / total if total > 0 else 0
#             train_acc_history.append(train_acc)
#
#             # 验证
#             model.set_train(False)
#             correct = 0
#             total = 0
#
#             # 只取2个验证批次
#             val_count = 0
#             for images, labels in test_dataset.create_tuple_iterator():
#                 outputs = model(images)
#                 predictions = ms.ops.argmax(outputs, 1)
#                 correct += (predictions == labels).sum().asnumpy()
#                 total += labels.shape[0]
#
#                 val_count += 1
#                 if val_count >= 2:
#                     break
#
#             val_acc = correct / total if total > 0 else 0
#             val_acc_history.append(val_acc)
#
#             if val_acc > best_val_acc:
#                 best_val_acc = val_acc
#
#             if (epoch + 1) % 2 == 0:
#                 print(f"  Epoch {epoch+1}/{epochs}: "
#                       f"训练Acc={train_acc:.3f}, 验证Acc={val_acc:.3f}")
#
#         results[name] = {
#             'params': info['params'],
#             'type': info['type'],
#             'train_acc_history': train_acc_history,
#             'val_acc_history': val_acc_history,
#             'best_val_acc': best_val_acc,
#             'final_train_acc': train_acc_history[-1],
#             'final_val_acc': val_acc_history[-1]
#         }
#
#         print(f"  ✅ 完成! 最佳验证Acc: {best_val_acc:.3f}")
#
#     return results
#
# def analyze_experiment_results(results):
#     """分析实验结果"""
#     print("\n" + "="*70)
#     print("📊 实验结果分析")
#     print("="*70)
#
#     # 创建对比表格
#     print("\n📈 性能对比表：")
#     print("-"*80)
#     print(f"{'模型':<15} {'参数量':<12} {'类型':<20} {'最终训练Acc':<12} {'最终验证Acc':<12} {'提升幅度':<10}")
#     print("-"*80)
#
#     base_val = results['base']['final_val_acc']
#
#     for name in ['base', 'wider', 'deeper', 'wider_deeper']:
#         if name in results:
#             r = results[name]
#             improvement = (r['final_val_acc'] - base_val) * 100 if name != 'base' else 0
#             print(f"{name:<15} {r['params']:<12,} {r['type']:<20} "
#                   f"{r['final_train_acc']:<12.3f} {r['final_val_acc']:<12.3f} "
#                   f"{improvement:>+6.1f}%" if name != 'base' else
#                   f"{name:<15} {r['params']:<12,} {r['type']:<20} "
#                   f"{r['final_train_acc']:<12.3f} {r['final_val_acc']:<12.3f} {'基准':>10}")
#
#     print("-"*80)
#
#     # 分析哪种策略更有效
#     print("\n🔬 有效性分析：")
#
#     wider_improvement = results['wider']['final_val_acc'] - base_val
#     deeper_improvement = results['deeper']['final_val_acc'] - base_val
#     wider_deeper_improvement = results['wider_deeper']['final_val_acc'] - base_val
#
#     print(f"1. 只增加通道数：验证Acc提升 {wider_improvement*100:.1f}%")
#     print(f"2. 只增加层数：验证Acc提升 {deeper_improvement*100:.1f}%")
#     print(f"3. 通道数+层数：验证Acc提升 {wider_deeper_improvement*100:.1f}%")
#
#     # 计算效率比（提升/参数增加比例）
#     wider_param_ratio = results['wider']['params'] / results['base']['params']
#     deeper_param_ratio = results['deeper']['params'] / results['base']['params']
#     wider_deeper_param_ratio = results['wider_deeper']['params'] / results['base']['params']
#
#     wider_efficiency = wider_improvement / (wider_param_ratio - 1) if wider_param_ratio > 1 else 0
#     deeper_efficiency = deeper_improvement / (deeper_param_ratio - 1) if deeper_param_ratio > 1 else 0
#     wider_deeper_efficiency = wider_deeper_improvement / (wider_deeper_param_ratio - 1) if wider_deeper_param_ratio > 1 else 0
#
#     print(f"\n📊 效率分析（单位参数增加带来的准确率提升）：")
#     print(f"  增加通道数效率: {wider_efficiency:.4f}")
#     print(f"  增加层数效率: {deeper_efficiency:.4f}")
#     print(f"  两者都增加效率: {wider_deeper_efficiency:.4f}")
#
#     # 给出建议
#     print("\n" + "="*70)
#     print("💡 实验结论与建议")
#     print("="*70)
#
#     if wider_efficiency > deeper_efficiency and wider_efficiency > wider_deeper_efficiency:
#         print("✅ **结论：增加通道数更有效**")
#         print("   原因：")
#         print("   1. 每增加1%参数，准确率提升更多")
#         print("   2. 更宽的模型能学习更多特征")
#         print("   3. 适合当前相对浅的网络结构")
#         print("\n  建议：优先增加通道数（16→32→64）")
#
#     elif deeper_efficiency > wider_efficiency and deeper_efficiency > wider_deeper_efficiency:
#         print("✅ **结论：增加层数更有效**")
#         print("   原因：")
#         print("   1. 增加深度能学习更抽象的特征")
#         print("   2. 深度对复杂任务更重要")
#         print("   3. 避免过宽导致的冗余")
#         print("\n  建议：优先增加层数（3层→4-5层）")
#
#     else:
#         print("✅ **结论：通道数和层数都需要增加**")
#         print("   原因：")
#         print("   1. 两者结合效果最好")
#         print("   2. 宽度和深度都很重要")
#         print("   3. 综合提升表达能力")
#         print("\n  建议：适度增加通道数（16→24）和层数（+1层）")
#
#     # 可视化结果
#     print("\n正在生成对比图表...")
#     import matplotlib.pyplot as plt
#
#     fig, axes = plt.subplots(2, 2, figsize=(14, 10))
#
#     # 1. 验证准确率对比
#     model_names = ['基准', '增加通道数', '增加层数', '两者都增加']
#     final_val_accs = [
#         results['base']['final_val_acc'],
#         results['wider']['final_val_acc'],
#         results['deeper']['final_val_acc'],
#         results['wider_deeper']['final_val_acc']
#     ]
#
#     colors = ['gray', 'blue', 'orange', 'green']
#     bars = axes[0, 0].bar(model_names, final_val_accs, color=colors, alpha=0.7)
#     axes[0, 0].set_ylabel('验证准确率')
#     axes[0, 0].set_title('不同策略的最终验证准确率')
#     axes[0, 0].set_ylim([0, 1])
#     for bar, val in zip(bars, final_val_accs):
#         axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
#                        f'{val:.3f}', ha='center', fontsize=10)
#
#     # 2. 训练曲线对比
#     epochs = range(1, 46)
#     axes[0, 1].plot(epochs, results['base']['val_acc_history'], 'k--', label='基准', alpha=0.7)
#     axes[0, 1].plot(epochs, results['wider']['val_acc_history'], 'b-', label='增加通道数', linewidth=2)
#     axes[0, 1].plot(epochs, results['deeper']['val_acc_history'], 'orange', label='增加层数', linewidth=2)
#     axes[0, 1].plot(epochs, results['wider_deeper']['val_acc_history'], 'g-', label='两者都增加', linewidth=2)
#     axes[0, 1].set_xlabel('Epoch')
#     axes[0, 1].set_ylabel('验证准确率')
#     axes[0, 1].set_title('验证准确率学习曲线')
#     axes[0, 1].legend()
#     axes[0, 1].grid(True, alpha=0.3)
#
#     # 3. 参数效率对比
#     param_ratios = [1, wider_param_ratio, deeper_param_ratio, wider_deeper_param_ratio]
#     improvements = [0, wider_improvement*100, deeper_improvement*100, wider_deeper_improvement*100]
#
#     ax3 = axes[1, 0]
#     scatter = ax3.scatter(param_ratios, improvements,
#                          s=[100, 200, 200, 300],
#                          c=colors, alpha=0.7)
#     ax3.set_xlabel('参数增加倍数 (相对于基准)')
#     ax3.set_ylabel('准确率提升 (%)')
#     ax3.set_title('参数效率分析')
#     ax3.grid(True, alpha=0.3)
#
#     # 添加标签
#     for i, txt in enumerate(model_names):
#         ax3.annotate(txt, (param_ratios[i], improvements[i]),
#                     xytext=(5, 5), textcoords='offset points')
#
#     # 4. 过拟合程度对比
#     gaps = []
#     for name in ['base', 'wider', 'deeper', 'wider_deeper']:
#         train_acc = results[name]['final_train_acc']
#         val_acc = results[name]['final_val_acc']
#         gaps.append(train_acc - val_acc)
#
#     bars = axes[1, 1].bar(model_names, gaps, color=colors, alpha=0.7)
#     axes[1, 1].set_ylabel('训练-验证差距')
#     axes[1, 1].set_title('过拟合程度对比')
#     axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
#
#     for bar, gap in zip(bars, gaps):
#         color = 'red' if gap > 0.1 else 'green'
#         axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
#                        f'{gap:.3f}', ha='center', fontsize=10, color=color)
#
#     plt.tight_layout()
#     plt.savefig('model_strategy_comparison.png', dpi=150, bbox_inches='tight')
#     plt.show()
#
#     print(f"\n✅ 对比图表已保存为 'model_strategy_comparison.png'")
#
# # 执行实验
# print("准备对比实验...")
#
# # 1. 创建数据（复用之前的数据集）
# print("\n加载数据...")
# train_dir = os.path.join(config.data_dir, "train")
# test_dir = os.path.join(config.data_dir, "test")
#
# # 简化变换用于快速实验
# simple_transform = [
#     vision.Decode(),
#     vision.Resize(config.image_size),
#     vision.ToTensor(),
#     vision.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], is_hwc=False)
# ]

# def create_small_dataset_for_test(data_path, batch_size=16):
#     dataset = ds.ImageFolderDataset(
#         data_path,
#         shuffle=True,
#         extensions=[".jpg", ".jpeg", ".png"]
#     )
#
#     dataset = dataset.map(simple_transform, input_columns="image")
#     dataset = dataset.map(transforms.TypeCast(ms.int32), input_columns="label")
#     dataset = dataset.batch(batch_size)
#
#     return dataset
#
# train_small = create_small_dataset_for_test(train_dir, batch_size=16)
# test_small = create_small_dataset_for_test(test_dir, batch_size=16)
#
# print(f"训练集批次: {train_small.get_dataset_size()}")
# print(f"测试集批次: {test_small.get_dataset_size()}")
#
# # 2. 创建对比模型
# models = create_models_for_testing()
#
# # 3. 运行快速实验
# results = run_quick_experiment(models, train_small, test_small, epochs=45)
#
# # 4. 分析结果
# analyze_experiment_results(results)
#
# print("\n" + "="*70)
# print("🎯 下一步行动")
# print("="*70)
# print("根据实验结果，选择最有效的策略修改你的模型，然后重新训练！")

In [ ]:
# test_model_capacity()